<a href="https://colab.research.google.com/github/d556645fw/Side-project/blob/main/%E6%99%BA%E6%85%A7%E5%AE%A2%E6%9C%8D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [2]:
import sqlite3

conn = sqlite3.connect('faq.db')
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS faq (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    question TEXT,
    answer TEXT,
    tags TEXT
)
''')

faqs = [
    ("如何重設密碼？", "請至設定 > 安全性中選擇重設密碼。", "帳號,密碼"),
    ("客服的聯絡方式是什麼？", "您可以透過 email@example.com 聯絡客服。", "聯絡,客服"),
    ("如何取消訂閱？", "請至帳號設定頁面中選擇取消訂閱。", "訂閱,取消"),
    ("有沒有提供退款？", "我們提供7天內退款保障，請聯絡客服。", "退款,保證")
]

cursor.executemany("INSERT INTO faq (question, answer, tags) VALUES (?, ?, ?)", faqs)
conn.commit()

In [3]:
from sentence_transformers import SentenceTransformer, util

# 使用中文效果不錯的模型
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

def get_faqs():
    cursor.execute("SELECT question, answer FROM faq")
    return cursor.fetchall()

def find_best_match(user_question, faqs, top_k=1):
    questions = [q for q, _ in faqs]
    embeddings = model.encode(questions + [user_question], convert_to_tensor=True)

    # 最後一個是 user_question
    similarity = util.cos_sim(embeddings[-1], embeddings[:-1])
    best_idx = similarity.argmax()
    score = similarity[0][best_idx].item()

    # 設定最低相似度門檻
    if score < 0.4:
        return "找不到相關答案，請聯絡客服進一步查詢。"
    return faqs[best_idx][1]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
while True:
    user_input = input("請輸入問題（輸入 'exit' 離開）：")
    if user_input.lower() == 'exit':
        break
    faqs = get_faqs()
    answer = find_best_match(user_input, faqs)
    print("AI 客服回答：", answer)

請輸入問題（輸入 'exit' 離開）：你好
AI 客服回答： 找不到相關答案，請聯絡客服進一步查詢。
請輸入問題（輸入 'exit' 離開）：請問密碼如何改
AI 客服回答： 請至設定 > 安全性中選擇重設密碼。
請輸入問題（輸入 'exit' 離開）：密碼怎改
AI 客服回答： 請至設定 > 安全性中選擇重設密碼。
請輸入問題（輸入 'exit' 離開）：密碼
AI 客服回答： 請至設定 > 安全性中選擇重設密碼。
請輸入問題（輸入 'exit' 離開）：密碼 退款
AI 客服回答： 請至設定 > 安全性中選擇重設密碼。
請輸入問題（輸入 'exit' 離開）：聯絡方式
AI 客服回答： 您可以透過 email@example.com 聯絡客服。
請輸入問題（輸入 'exit' 離開）：exit
